In [80]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np

from word2vec import Word2Vec, Sent2Vec
import nltk
from nltk.corpus import stopwords
from gensim import corpora
import re

## Connection MongoDB

In [81]:
from pymongo import MongoClient
client = MongoClient()

In [82]:
db = client.msbd500103
print db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'msbd500103')


## Training Data

In [83]:
trData = db.training_record
print trData.count()
pprint.pprint(trData.find_one())

178484
{u'_id': ObjectId('5a07224d6c0c65269105e6c7'),
 u'article_contentid': u'1l5c17e72og_us',
 u'article_lan': u'en',
 u'dt': u'20170111',
 u'filter': None,
 u'user_age': 2,
 u'user_all_pkgs': [u'com.cvs.launchers.cvs',
                    u'com.flexplan.flexi',
                    u'com.cleanmaster.security',
                    u'com.dmdbrands.gurus.weight',
                    u'com.gotv.crackle.handset',
                    u'com.oprah.owntve',
                    u'com.qihoo.batterysaverplus',
                    u'com.everalbum.everalbumapp',
                    u'com.google.android.apps.plus',
                    u'com.sec.android.app.clockpackage',
                    u'com.android.vending',
                    u'com.cleanmaster.mguard',
                    u'com.google.android.gm',
                    u'jakiganicsystems.simplestchecklist',
                    u'com.google.android.googlequicksearchbox',
                    u'com.froogloid.kring.google.zxing.client.android',
 

### Data Filtering

In [84]:
allData = trData.find({})
data = list(allData)
df = pd.DataFrame(data)
df_all = df.dropna(subset=['user_gp_frequency','article_title'])
df_all = df_all.reset_index(drop=True)
df_all.head()

,_id,article_contentid,article_dwelltime,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time,dt,...,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a07224d6c0c65269105e6c9,Ly7407d4cuC_us,31.0,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988...",en,The Sun,"Dad caught selling his baby son for £9,000 to ...",1.484656e+09,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
1,5a07224d6c0c65269105e6ca,AY628126a1w_us,NaN,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]",en,Bustle,11 Weird Ways Your Relationship With Your Pare...,1.482379e+09,20170110,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
2,5a07224d6c0c65269105e6cb,jv70536a3bt_us,NaN,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895...",en,Los Angeles Times,Ivanka Trump goes public about exiting fashion...,1.484223e+09,20170113,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
3,5a07224d6c0c65269105e6cc,1V70d401fdc_us,NaN,"[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765...",en,Footwear News,Michelle Obama Wears Edgy Look for Final Talk ...,1.484261e+09,20170112,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
4,5a07224d6c0c65269105e6cd,jQ6c581bbr2_us,159.0,"[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718...",en,Most Trending News,Hehe! See this bedroom pictures of couple that...,1.483782e+09,20170110,...,US,2.0,0.852654,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR...",NaN,CD2F2B61135DB27BA3230F91466ED793,Lebanon,US,PA,SAMSUNG-SM-G890A


In [85]:
df.article_dwelltime.nunique()

1612

In [86]:
df_all.shape
#row = range(1, df_all.shape[0])
#df_all = df_all.reindex(row)
#df_all.head()

(168276, 24)

### User Features 

In [87]:
user_cnt = df_all['user_country'].unique()
print user_cnt
#-> don't use country feature

user_cty = df_all['user_maxmind_city'].unique()
print user_cty, len(user_cty)
#-> can use city feature

[u'US']
[u'Glen Allen' u'Lebanon' u'Concord' ..., u'Potomac' u'Bloomfield'
 u'Buchanan'] 2307


In [88]:
user_gend = df_all['user_gender'].unique()
print user_gend
#-> treat nan as one gender feature

user_age = df_all['user_age'].unique()
print user_age
#-> treat nan as one gender feature

temp = df_all[df_all['user_age'].isnull()][['user_id']]
temp_uid = temp['user_id'].unique()
print temp.count()
print len(temp_uid)
#-> treat nan as one age feature

[  2.   1.  nan]
[  2.   3.   4.   1.  nan]
user_id    10264
dtype: int64
590


In [89]:
user_mdl = df_all['user_model'].unique()
print len(user_mdl)
#-> can use model (may or may not)

user_brd = df_all['user_brand'].unique()
print len(user_brd)
#-> can use brand

649
50


In [90]:
df_user = df_all[['user_id','user_gender','user_age',\
                  'user_model','user_brand','user_maxmind_city',\
                  'user_gp_frequency']]
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user = df_user.reset_index(drop=True)
df_user.head()


,user_id,user_gender,user_age,user_model,user_brand,user_maxmind_city,user_gp_frequency
0,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,SPH-L710,samsung,Glen Allen,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
1,CD2F2B61135DB27BA3230F91466ED793,2.0,3.0,SAMSUNG-SM-G890A,samsung,Lebanon,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR..."
2,56F27AD705D8564F7900470AA793DBD2,2.0,4.0,E6782,kyocera,Concord,"{u'COMMUNICATION': 1, u'TOOLS': 3, u'GAME_CASU..."
3,8EE461BA3AA2595AC91FCDD833EC1711,2.0,2.0,SM-G920R4,samsung,Aurora,"{u'GAME_CARD': 1, u'SHOPPING': 2, u'FINANCE': ..."
4,5BA72C11B1E94BA50F1E66F4B1362CE6,1.0,2.0,SM-T217S,samsung,Fort Wayne,"{u'HOUSE_AND_HOME': 1, u'SHOPPING': 1, u'FINAN..."


In [91]:
def dealWstring(string):
    string = string.lower()
    return string
df_user['user_brand'] = df_user.apply(lambda row: dealWstring(row['user_brand']),axis = 1)
df_user.head()

,user_id,user_gender,user_age,user_model,user_brand,user_maxmind_city,user_gp_frequency
0,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,SPH-L710,samsung,Glen Allen,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
1,CD2F2B61135DB27BA3230F91466ED793,2.0,3.0,SAMSUNG-SM-G890A,samsung,Lebanon,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR..."
2,56F27AD705D8564F7900470AA793DBD2,2.0,4.0,E6782,kyocera,Concord,"{u'COMMUNICATION': 1, u'TOOLS': 3, u'GAME_CASU..."
3,8EE461BA3AA2595AC91FCDD833EC1711,2.0,2.0,SM-G920R4,samsung,Aurora,"{u'GAME_CARD': 1, u'SHOPPING': 2, u'FINANCE': ..."
4,5BA72C11B1E94BA50F1E66F4B1362CE6,1.0,2.0,SM-T217S,samsung,Fort Wayne,"{u'HOUSE_AND_HOME': 1, u'SHOPPING': 1, u'FINAN..."


In [92]:
df_user.shape
df_user.user_id.nunique()

7857

In [95]:
df_userid = df_user[['user_id']]
df_userfea = df_user[['user_gender','user_age',\
                  'user_model','user_brand']]
df_appcat = df_user[['user_id','user_gp_frequency']]
df_userfea.head()

,user_gender,user_age,user_model,user_brand
0,2.0,2.0,SPH-L710,samsung
1,2.0,3.0,SAMSUNG-SM-G890A,samsung
2,2.0,4.0,E6782,kyocera
3,2.0,2.0,SM-G920R4,samsung
4,1.0,2.0,SM-T217S,samsung


In [96]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_userfea_2 = df_userfea
df_userfea_2 = df_userfea.apply(le.fit_transform)
df_userfea_2.head()

,user_gender,user_age,user_model,user_brand
0,1,1,550,28
1,1,2,319,28
2,1,3,91,17
3,1,1,420,28
4,0,1,511,28


In [97]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(df_userfea_2)
onehotlabels = enc.transform(df_userfea_2).toarray()
onehotlabels.shape
print type(onehotlabels)

<type 'numpy.ndarray'>


In [98]:
print type(onehotlabels)
print onehotlabels
onehot_usrfea = pd.DataFrame(onehotlabels)
onehot_usrfea.head()

<type 'numpy.ndarray'>
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


,0,1,2,3,4,5,6,7,8,9,...,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
user_merge = df_userid.merge(onehot_usrfea, how = 'left', left_index=True, right_index = True)
user_merge.head()

,user_id,0,1,2,3,4,5,6,7,8,...,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075
0,614E304DE3B07EBCDD4E11382578EF5A,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CD2F2B61135DB27BA3230F91466ED793,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,56F27AD705D8564F7900470AA793DBD2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8EE461BA3AA2595AC91FCDD833EC1711,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5BA72C11B1E94BA50F1E66F4B1362CE6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### User App Profile

App Package list: most frequent item 99979

In [100]:
all_pkgs = list()
for i in df_all['user_all_pkgs']:
    all_pkgs.extend(i)
dict_pkgs = dict(Counter(all_pkgs))
#sorted(dict_pkgs.items(),key=lambda x:x[1], reverse=True)
#len(list(set(all_pkgs)))

In [101]:
all_category = list()
for i in df_appcat['user_gp_frequency']:
    all_category.extend(i)
dict_category = dict(Counter(all_category))
#dict_category = sorted(dict_category.items(),key=lambda x:x[1], reverse=True)
dict_category

{u'ART_AND_DESIGN': 116,
 u'AUTO_AND_VEHICLES': 95,
 u'BEAUTY': 56,
 u'BOOKS_AND_REFERENCE': 2254,
 u'BUSINESS': 1611,
 u'COMICS': 112,
 u'COMMUNICATION': 5879,
 u'DATING': 155,
 u'EDUCATION': 1577,
 u'ENTERTAINMENT': 4791,
 u'EVENTS': 30,
 u'FAMILY_ACTION': 2,
 u'FINANCE': 2714,
 u'FOOD_AND_DRINK': 595,
 u'GAME_ACTION': 1438,
 u'GAME_ADVENTURE': 1304,
 u'GAME_ARCADE': 1645,
 u'GAME_BOARD': 640,
 u'GAME_CARD': 1214,
 u'GAME_CASINO': 828,
 u'GAME_CASUAL': 2194,
 u'GAME_EDUCATIONAL': 413,
 u'GAME_MUSIC': 162,
 u'GAME_PUZZLE': 2072,
 u'GAME_RACING': 727,
 u'GAME_ROLE_PLAYING': 589,
 u'GAME_SIMULATION': 1125,
 u'GAME_SPORTS': 1024,
 u'GAME_STRATEGY': 1098,
 u'GAME_TRIVIA': 385,
 u'GAME_WORD': 788,
 u'HEALTH_AND_FITNESS': 1867,
 u'HOUSE_AND_HOME': 473,
 u'LIBRARIES_AND_DEMO': 86,
 u'LIFESTYLE': 2990,
 u'MAPS_AND_NAVIGATION': 1479,
 u'MEDIA_AND_VIDEO': 12,
 u'MEDICAL': 377,
 u'MUSIC_AND_AUDIO': 4634,
 u'NEWS_AND_MAGAZINES': 3248,
 u'PARENTING': 46,
 u'PERSONALIZATION': 1881,
 u'PHOTOGRAPHY':

In [102]:
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i not in set(y) ])
unwanted = set()
unwanted = {'TOOLS', 'COMMUNICATION', 'SOCIAL', 'PRODUCTIVITY',\
            'VIDEO_PLAYERS', 'ENTERTAINMENT','MUSIC_AND_AUDIO', \
            'TRAVEL_AND_LOCAL', 'PHOTOGRAPHY', 'NEWS_AND_MAGAZINES',\
            'WEATHER','MAPS_AND_NAVIGATION'}
wantcat = set(dict_category.keys()) - unwanted

In [103]:
df_appcat.head()

,user_id,user_gp_frequency
0,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
1,CD2F2B61135DB27BA3230F91466ED793,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR..."
2,56F27AD705D8564F7900470AA793DBD2,"{u'COMMUNICATION': 1, u'TOOLS': 3, u'GAME_CASU..."
3,8EE461BA3AA2595AC91FCDD833EC1711,"{u'GAME_CARD': 1, u'SHOPPING': 2, u'FINANCE': ..."
4,5BA72C11B1E94BA50F1E66F4B1362CE6,"{u'HOUSE_AND_HOME': 1, u'SHOPPING': 1, u'FINAN..."


In [104]:
def getValue(dic, cat):
    if (cat in dic.keys()) == 0:
        return 0
    else:
        return 1
a = getValue(df_appcat['user_gp_frequency'][0], 'TY')
                    

In [105]:
for cat in wantcat:
    cat = str(cat)
    df_appcat[cat] = df_appcat.apply(lambda row: getValue(row['user_gp_frequency'], cat), axis=1)
df_appcat.head()


/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,user_gp_frequency,LIBRARIES_AND_DEMO,LIFESTYLE,BUSINESS,MEDIA_AND_VIDEO,MEDICAL,GAME_SPORTS,BOOKS_AND_REFERENCE,GAME_EDUCATIONAL,...,GAME_STRATEGY,GAME_MUSIC,GAME_ARCADE,GAME_CASINO,GAME_BOARD,SPORTS,BEAUTY,GAME_SIMULATION,EDUCATION,FOOD_AND_DRINK
0,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,CD2F2B61135DB27BA3230F91466ED793,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,56F27AD705D8564F7900470AA793DBD2,"{u'COMMUNICATION': 1, u'TOOLS': 3, u'GAME_CASU...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8EE461BA3AA2595AC91FCDD833EC1711,"{u'GAME_CARD': 1, u'SHOPPING': 2, u'FINANCE': ...",0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5BA72C11B1E94BA50F1E66F4B1362CE6,"{u'HOUSE_AND_HOME': 1, u'SHOPPING': 1, u'FINAN...",0,0,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


### Training Article List

In [106]:
df_art = df_all[df_all['article_contentid'].notnull()][['article_contentid','article_title',\
                  'article_l1_categories','article_l2_categories']]
df_art = df_art.drop_duplicates(subset=['article_contentid'], keep='first')
df_art = df_art.reset_index(drop=True)
df_art.head()

,article_contentid,article_title,article_l1_categories,article_l2_categories
0,Ly7407d4cuC_us,"Dad caught selling his baby son for £9,000 to ...","[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988..."
1,AY628126a1w_us,11 Weird Ways Your Relationship With Your Pare...,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]"
2,jv70536a3bt_us,Ivanka Trump goes public about exiting fashion...,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895..."
3,1V70d401fdc_us,Michelle Obama Wears Edgy Look for Final Talk ...,"[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765..."
4,jQ6c581bbr2_us,Hehe! See this bedroom pictures of couple that...,"[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718..."


In [113]:
tmp = list(df_art.article_l1_categories)
pd.DataFrame(tmp)
print type(tmp[16])
print type(tmp[1][0])

<type 'list'>
<type 'dict'>


In [114]:
#nltk.download('stopwords')
#print stopwords.words('english')
def dealWstring1(string):
    #string = str(liststr).replace("!@#$%^&*()[]{};:,./<>?\|`~-=_+", " ")
    #string = str(liststr)
    string = re.sub('[^a-zA-Z0-9 \n\.]', '', string)
    word_list = string.strip().lower().split()
    filtered_words = [word for word in word_list if word not in stopwords.words('english')]
    return filtered_words

In [115]:
df_art['article_title'] = df_art.apply(lambda row: dealWstring1(row['article_title']),axis = 1)
df_art.head()

,article_contentid,article_title,article_l1_categories,article_l2_categories
0,Ly7407d4cuC_us,"[dad, caught, selling, baby, son, 9000, buy, c...","[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988..."
1,AY628126a1w_us,"[11, weird, ways, relationship, parents, affec...","[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]"
2,jv70536a3bt_us,"[ivanka, trump, goes, public, exiting, fashion...","[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895..."
3,1V70d401fdc_us,"[michelle, obama, wears, edgy, look, final, ta...","[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765..."
4,jQ6c581bbr2_us,"[hehe, see, bedroom, pictures, couple, everyon...","[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718..."


In [116]:
dictionary = corpora.Dictionary(df_art['article_title'])
word_dic = dictionary.token2id
word_dic

{u'belligerence': 21021,
 u'indianamericans': 32098,
 u'ampitheatre': 37534,
 u'vani': 28146,
 u'woods': 1042,
 u'exorcists': 34152,
 u'spiders': 861,
 u'foluke': 20412,
 u'hanging': 296,
 u'woody': 5696,
 u'comically': 17459,
 u'canes': 30926,
 u'gamethread': 31406,
 u'brockton': 3810,
 u'antidandruff': 28087,
 u'caned': 30208,
 u'fourcar': 21926,
 u'programsof': 23056,
 u'offduty': 8846,
 u'kaspersky': 24117,
 u'exchief': 35559,
 u'appropriation': 27237,
 u'holyrood': 19305,
 u'screaming': 2582,
 u'showroom': 32897,
 u'advices': 19957,
 u'grueling': 34184,
 u'wooden': 20184,
 u'wednesday': 3359,
 u'chameleons': 29764,
 u'crotch': 1520,
 u'stereotypical': 25437,
 u'bbqs': 23592,
 u'chatrice': 27669,
 u'snuggles': 32982,
 u'francesco': 20643,
 u'scrapes': 23221,
 u'shorthaul': 33342,
 u'miniatures': 29839,
 u'272': 10327,
 u'magnitude2.6': 35635,
 u'275': 28847,
 u'imploding': 14129,
 u'scraped': 17807,
 u'glossip': 37065,
 u'starboy': 19853,
 u'arrowverse': 25525,
 u'12017': 25541,
 u

In [117]:
print len(word_dic)
def word2id(dic, filtered):
    code = [word_dic[word] for word in filtered]
    return code

38223


In [118]:
df_art['article_title'] = df_art.apply(lambda row: word2id(word_dic, row['article_title']),axis = 1)
df_art.head()

,article_contentid,article_title,article_l1_categories,article_l2_categories
0,Ly7407d4cuC_us,"[0, 4, 1, 9, 6, 12, 7, 3, 5, 2, 10, 11, 8]","[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988..."
1,AY628126a1w_us,"[13, 20, 17, 16, 18, 19, 15, 14]","[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]"
2,jv70536a3bt_us,"[26, 22, 24, 28, 23, 21, 27, 22, 25]","[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895..."
3,1V70d401fdc_us,"[29, 35, 30, 34, 31, 37, 38, 32, 33, 39, 36]","[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765..."
4,jQ6c581bbr2_us,"[47, 45, 46, 42, 43, 40, 44, 41]","[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718..."


In [73]:
def list2one(dic, filtered):
    code = [word_dic[word] for word in filtered]
    return code

In [119]:
##don't run: time cost
'''
word_set = sorted(word_dic.values())
for wid in range(2):
    name = 'word'+str(wid)
    wid = int(wid)
    df_art[name] = df_art.apply(lambda row: getValue(row['article_title'], cat), axis=1)
''''''

AttributeError: ("'int' object has no attribute 'keys'", u'occurred at index 0')

In [120]:
df_art.shape

(49149, 4)

In [197]:
def get1Name(cat):
    if len(cat[1]) == 0:
        return 'nan'
        print 'nan'
    return cat[1][0]['name']

def get2Name(cat):
    if len(cat[2]) == 0:
        return 'nan'
        print 'nan'
    return cat[2][0]['name']

#getName(df_artCat.iloc[1896]['article_l2_categories'][0])

In [198]:
df_artCat.iloc[1896]['article_l1_categories']

[]

In [199]:
df_artCat = df_art[['article_contentid','article_l1_categories','article_l2_categories']]
df_artCat.head()

,article_contentid,article_l1_categories,article_l2_categories
0,Ly7407d4cuC_us,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988..."
1,AY628126a1w_us,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]"
2,jv70536a3bt_us,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895..."
3,1V70d401fdc_us,"[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765..."
4,jQ6c581bbr2_us,"[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718..."


In [200]:
type(df_artCat.iloc[0]['article_l1_categories'])

list

In [201]:
df_artCat['l1_categories'] = df_artCat.apply(lambda row: get1Name(row), axis=1)
df_artCat['l2_categories'] = df_artCat.apply(lambda row: get2Name(row), axis=1)

In [202]:
df_artCat.head()

,article_contentid,article_l1_categories,article_l2_categories,l1_categories,l2_categories
0,Ly7407d4cuC_us,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988...",1000780,1000667
1,AY628126a1w_us,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]",1000288,1000288
2,jv70536a3bt_us,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895...",1000031,1000069
3,1V70d401fdc_us,"[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765...",1000031,1000069
4,jQ6c581bbr2_us,"[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718...",1001091,1001091


### Sen2Vec

In [207]:
from gensim.models import word2vec
sentences = word2vec.Text8Corpus('text8')
model = word2vec.Word2Vec(sentences, size=200)

IOError: [Errno 2] No such file or directory: 'text8'

### TF-IDF

### Record: User-id match article-id

In [250]:
df_map = df[['user_id', 'article_contentid']]
df_map['read']=1
df_map.head()

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,article_contentid,read
0,614E304DE3B07EBCDD4E11382578EF5A,1l5c17e72og_us,1
1,614E304DE3B07EBCDD4E11382578EF5A,mQ56173a2KT_us,1
2,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,1
3,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,1
4,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,1


### Test Article

In [91]:
tstArt = db.test_article
pprint.pprint(tstArt.find_one())

{u'_id': ObjectId('5a0722856c0c65269108a0b6'),
 u'article_contentid': u'sq7f5162eGP_us',
 u'article_l1_categories': [{u'name': u'1000560',
                             u'weight': 0.7253321861848524}],
 u'article_l2_categories': [{u'name': u'1000596',
                             u'weight': 0.9452144284738275}],
 u'article_lan': u'en',
 u'article_publisher': u'Spasique',
 u'article_title': u'Nastiest Thing You Are Willing To Do In Bed, Based on Your Zodiac Sign',
 u'article_update_time': 1485686119}


In [92]:
allArtData = tstArt.find({})
artdata = list(allArtData)
df_tmp = pd.DataFrame(artdata)
df_tmp = df_tmp.dropna(subset=['article_title'])

In [93]:
df_tmp['article_title'] = df_tmp.apply(lambda row: dealWstring1(row['article_title']),axis = 1)

In [94]:
df_tmp.head()

,_id,article_contentid,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time
0,5a0722856c0c65269108a0b6,sq7f5162eGP_us,"[{u'name': u'1000560', u'weight': 0.7253321861...","[{u'name': u'1000596', u'weight': 0.9452144284...",en,Spasique,"[nastiest, thing, willing, bed,, based, zodiac...",1.485686e+09
1,5a0722856c0c65269108a0b7,dh7f0517aP4_us,"[{u'name': u'1000031', u'weight': 0.8983825108...","[{u'name': u'1000069', u'weight': 0.7455885870...",en,Vulture,"[matthew, mcconaughey, anne, hathaway, talks, ...",1.485696e+09
2,5a0722856c0c65269108a0b8,AE78003f4fk_us,"[{u'name': u'1000031', u'weight': 0.9555669128...","[{u'name': u'1000031', u'weight': 0.9555669128...",en,The Hollywood Gossip,"[love, &, hip, hop, cast, rankings:, actually,...",1.485084e+09
3,5a0722856c0c65269108a0b9,oj7e81b559q_us,"[{u'name': u'1000031', u'weight': 0.9492414325...","[{u'name': u'1000069', u'weight': 0.7962460693...",en,LBCC Viking News,"[rasheeda, 'love, hip, hop, atlanta', responds...",1.485685e+09
4,5a0722856c0c65269108a0ba,h57baa257GX_us,"[{u'name': u'1000931', u'weight': 0.5076023340...","[{u'name': u'1000931', u'weight': 0.5076023340...",en,Smithsonian Channel,"[brave, diver, spots, giant, fish., camera, pa...",1.485714e+09
